<a href="https://colab.research.google.com/github/ewuerfel66/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/EricWuerfel_LS_DS5_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

# 1980's Congressional Voting Data

## Imports & Data

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)

In [0]:
df = df.rename(columns={
    0:'party'
    , 1:'handicapped-infants'
    , 2:'water-project-cost-sharing'
    , 3:'budget-resolution'
    , 4:'physician-fee-freeze'
    , 5:'el-salvador-aid'
    , 6:'religious-groups-in-schools'
    , 7:'anti-satellite-test-ban'
    , 8:'aid-to-nicaraguan-contras'
    , 9:'mx-missile'
    , 10:'immigration'
    , 11:'synfuels-corporation-cutback'
    , 12:'education-spending'
    , 13:'superfund-right-to-sue'
    , 14:'crime'
    , 15:'duty-free-exports'
    , 16:'export-administration-act-south-africa'
})

In [0]:
df = df.replace(to_replace="?", value = np.NaN)

In [0]:
df = df.replace(to_replace="n", value = 0)

In [0]:
df = df.replace(to_replace="y", value = 1)

In [0]:
df.head()

,party,handicapped-infants,water-project-cost-sharing,budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
df.isna().sum()

party                                       0
handicapped-infants                        12
water-project-cost-sharing                 48
budget-resolution                          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [0]:
# Split Data into Parties

rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

In [0]:
rep.head()

,party,handicapped-infants,water-project-cost-sharing,budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
dem.head()

,party,handicapped-infants,water-project-cost-sharing,budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


## Democrats support more than Republicans

$$H_0: \mu_{dems} = \mu_{reps}$$

$$H_A: \mu_{dems} > \mu_{reps}$$

In [0]:
from scipy.stats import ttest_ind

In [0]:
print("Democrat Support: ", dem['aid-to-nicaraguan-contras'].mean())
print("Republican Support: ", rep['aid-to-nicaraguan-contras'].mean())

Democrat Support:  0.8288973384030418
Republican Support:  0.15286624203821655


In [0]:
ttest_ind(dem['budget-resolution'], rep['budget-resolution'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

Our two-sided t-test returned a p-value that is Far Far Less than 0.01. The p-value for a one-sided test would be even lower than this. With this p-value we can REJECT THE NULL HYPOTHESIS with 99% confidence. A significantly higher proportion of democrats supported the budget resolution when compared with republicans.

## Republicans support more than Democrats

$$H_0: \mu_{reps} = \mu_{dems}$$

$$H_A: \mu_{reps} > \mu_{dems}$$

In [0]:
print("Democrat Support: ", dem['physician-fee-freeze'].mean())
print("Republican Support: ", rep['physician-fee-freeze'].mean())

Democrat Support:  0.05405405405405406
Republican Support:  0.9878787878787879


In [0]:
ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

Our two-sided t-test returned a p-value that is far less than 0.01. The p-value for a one-sided test would be even lower than this. With this p-value we can REJECT THE NULL HYPOTHESIS with 99% confidence. A significantly higher proportion of republicans support the physician fee freeze measure when compared with democrats.

## Similar Republican & Democratic Support

$$H_0: \mu_{reps} = \mu_{dems}$$

$$H_A: \mu_{reps} \neq \mu_{dems}$$

In [0]:
print("Democrat Support: ", dem['water-project-cost-sharing'].mean())
print("Republican Support: ", rep['water-project-cost-sharing'].mean())

Democrat Support:  0.502092050209205
Republican Support:  0.5067567567567568


In [0]:
ttest_ind(rep['water-project-cost-sharing'], dem['water-project-cost-sharing'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

Our two-sided t-test returned a p-vale that is far greater than 0.1. With this p-value, we FAIL TO REJECT THE NULL HYPOTHESIS and conclude that there is no significant difference between democratic and republican support for the water project cost sharing measure.

# Insulin Prices

## Data

In [0]:
df = pd.read_csv('https://www.healthcostinstitute.org/images/pdfs/171117_State_Insulin_Average_Point-of-Sale_Prices.csv')

In [0]:
df = df.rename(columns={'Average_Point-of-Sale_Price_per_Script':'ave_price'})

In [0]:
# Split the data
insulin_2012 = df[df['year'] == 2012];
insulin_2016 = df[df['year'] == 2016];

## Student's t-test

Let $\mu$ be **the average price of 10 mL, 100 units/mL for a given year**

$$H_0: \mu_{2012} = \mu_{2016}$$
$$H_A: \mu_{2012} < \mu_{2016}$$

In [54]:
print("2012: ", insulin_2012['ave_price'].mean())
print("2016: ", insulin_2016['ave_price'].mean())

2012:  345.15686274509807
2016:  687.4313725490196


In [55]:
ttest_ind(insulin_2012['ave_price'], insulin_2016['ave_price'], nan_policy='omit')

Ttest_indResult(statistic=-33.0922457991535, pvalue=1.1201823655624986e-55)

In [58]:
# With a high DoF (DoF = 50), the t-distribution approaches a Normal distribution
# which is a symetric distribution
pvalue = 1.120182365562498e-55 / 2
print('The p-value for my one-sided test is approximately: ', pvalue)

The p-value for my one-sided test is approximately:  5.60091182781249e-56


$$p-value = 5.6*10^{-56} << 0.01$$

Therefore, we REJECT THE NULL HYPOTHESIS that the average US price of Insulin has remained the same from 2012 to 2016. We can say with 99% confidence that the price of insulin in the US has increased significantly from 2012 to 2016.